In [2]:
import os
import pandas as pd
import numpy as np

# Create Spark context
from pyspark import SparkContext, SparkConf
from pyspark.sql import functions as F
from pyspark.sql.types import *
conf = SparkConf().setAppName('dj').setMaster('local[*]')
sc = SparkContext(conf=conf)

# Create Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('dj').getOrCreate()
spark

In [3]:
(sc.applicationId, sc.master, sc.version, sc.uiWebUrl, sc.defaultParallelism, sc.pythonVer, sc.appName, sc.sparkUser,sc,spark.catalog)

('local-1698602911007',
 'local[*]',
 '3.5.0',
 'http://USHYDJDAMODH6.us.deloitte.com:4040',
 16,
 '3.11',
 'dj',
 <bound method SparkContext.sparkUser of <SparkContext master=local[*] appName=dj>>,
 <SparkContext master=local[*] appName=dj>,
 <pyspark.sql.catalog.Catalog at 0x299efa54190>)

In [8]:
# Import the data to a DataFrame
departures_df = spark.read.csv('./AA_DFW_2015_Departures_Short.csv.gz', header=True)

# Remove any duration of 0
departures_df = departures_df\
    .filter(departures_df['Actual elapsed time (Minutes)'] == 0)

# Add an ID column
departures_df = departures_df.withColumn('id', F.monotonically_increasing_id())

# Write the file out to JSON format
departures_df.write.json('tmp_output.json', mode='overwrite')
departures_df.show(4,truncate=False)

+-----------------+-------------+-------------------+-----------------------------+---+
|Date (MM/DD/YYYY)|Flight Number|Destination Airport|Actual elapsed time (Minutes)|id |
+-----------------+-------------+-------------------+-----------------------------+---+
|01/01/2015       |0029         |ONT                |0                            |0  |
|01/01/2015       |0043         |DTW                |0                            |1  |
|01/01/2015       |0074         |CLE                |0                            |2  |
|01/01/2015       |0150         |MSY                |0                            |3  |
+-----------------+-------------+-------------------+-----------------------------+---+
only showing top 4 rows



In [5]:
# Import the file to a DataFrame and perform a row count
annotations_df = spark.read.csv('annotations.csv.gz', sep='|')
full_count = annotations_df.count()

# Count the number of rows beginning with '#'
comment_count = annotations_df.where(col('_c0').startswith('#')).count()

# Import the file to a new DataFrame, without commented rows
no_comments_df = spark.read.csv('annotations.csv.gz', sep='|', comment='#')

# Count the new DataFrame and verify the difference is as expected
no_comments_count = no_comments_df.count()
print("Full count: %d\nComment count: %d\nRemaining count: %d" % (full_count, comment_count, no_comments_count))

AnalysisException: [PATH_NOT_FOUND] Path does not exist: file:/c:/Users/jdamodhar/Desktop/python_essential/DataCamp-ds-deloitte-master/00-A/53-Cleaning Data with PySpark/annotations.csv.gz.

In [ ]:
# Split _c0 on the tab character and store the list in a variable
tmp_fields = F.split(annotations_df['_c0'], "\t")

# Create the colcount column on the DataFrame
annotations_df = annotations_df.withColumn('colcount', F.size(tmp_fields))

# Remove any rows containing fewer than 5 fields
annotations_df_filtered = annotations_df.filter(~ (annotations_df.colcount < 5))

# Count the number of rows
final_count = annotations_df_filtered.count()
print("Initial count: %d\nFinal count: %d" % (initial_count, final_count))

In [ ]:

# Remove any rows containing fewer than 5 fields
annotations_df_filtered = annotations_df.filter(~ (annotations_df.colcount < 5))

# Count the number of rows
final_count = annotations_df_filtered.count()
print("Initial count: %d\nFinal count: %d" % (initial_count, final_count))

In [ ]:
# Split the content of _c0 on the tab character (aka, '\t')
split_cols = F.split(annotations_df["_c0"], '\t')

# Add the columns folder, filename, width, and height
split_df = annotations_df.withColumn('folder', split_cols.getItem(0))
split_df = split_df.withColumn('filename', split_cols.getItem(1))
split_df = split_df.withColumn('width', split_cols.getItem(2))
split_df = split_df.withColumn('height', split_cols.getItem(3))

# Add split_cols as a column
split_df = split_df.withColumn('split_cols', split_cols)

In [ ]:
def retriever(cols, colcount):
  # Return a list of dog data
  return cols[4:colcount]

# Define the method as a UDF
udfRetriever = F.udf(retriever, ArrayType(StringType()))

# Create a new column using your UDF
split_df = split_df.withColumn(
    'dog_list',
    udfRetriever(split_df.split_cols, split_df.colcount)
)

# Remove the original column, split_cols, and the colcount
split_df = split_df\
    .drop('_c0')\
    .drop('split_cols')\
    .drop('colcount')

In [ ]:
# Rename the column in valid_folders_df
valid_folders_df = valid_folders_df.withColumnRenamed('_c0', 'folder')

# Count the number of rows in split_df
split_count = split_df.count()

# Join the DataFrames
joined_df = split_df.join(F.broadcast(valid_folders_df), "folder")

# Compare the number of rows remaining
joined_count = joined_df.count()
print("Before: %d\nAfter: %d" % (split_count, joined_count))

In [ ]:
# Determine the row counts for each DataFrame
split_count = split_df.count()
joined_count = joined_df.count()

# Create a DataFrame containing the invalid rows
invalid_df = split_df.join(F.broadcast(joined_df), 'folder', 'left_anti')

# Validate the count of the new DataFrame is as expected
invalid_count = invalid_df.count()
print(" split_df:\t%d\n joined_df:\t%d\n invalid_df: \t%d" % (split_count, joined_count, invalid_count))

# Determine the number of distinct folder columns removed
invalid_folder_count = invalid_df.select('folder').distinct().count()
print("%d distinct invalid folders found" % invalid_folder_count)